# Data Processing and Visualization with PostgreSQL and GeoPandas

This Jupyter Notebook demonstrates how to:
1. Connect to a PostgreSQL database.
2. Query data related to event counts, location counts, and user information.
3. Visualize the data with line plots, bar plots, and geospatial choropleth maps using `matplotlib`, `seaborn`, and `geopandas`.

---

### Step 1: Importing Required Libraries

Let's start by importing all the necessary libraries for database connection, data processing, and visualization.

In [1]:
# %%
# Importing necessary libraries
import os
import pandas as pd
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd

### Step 2: Setting Up Database Connection

The environment variables for database connection are retrieved, and we will create an engine to interact with the PostgreSQL database.

In [2]:
# %%
# Retrieving environment variables for database connection
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")
POSTGRES_HOST = os.getenv("POSTGRES_HOST")
POSTGRES_PORT = os.getenv("POSTGRES_PORT")
POSTGRES_DB = os.getenv("POSTGRES_DB")

# Constructing the PostgreSQL connection URL using the environment variables
database_url = f"postgresql://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}"
# Creating an engine to connect to the PostgreSQL database using SQLAlchemy
engine = create_engine(database_url)

### Step 3: Querying and Visualizing Event Data

We will query the database for the number of events per hour and visualize it with a line plot.

In [3]:
# %%
# Querying the database for the number of events per hour
query_events_per_hour = "SELECT * FROM events_per_hour;"
df_events_per_hour = pd.read_sql_query(query_events_per_hour, engine)

# %%
# Visualizing the event counts per hour using a line plot
plt.figure(figsize=(12, 6))
sns.set_style("whitegrid")  # Setting the style for the plot

sns.lineplot(data=df_events_per_hour, x='hour', y='event_count', marker='o')

# Labels and formatting for the plot
plt.xlabel("Hour")
plt.ylabel("Event Count")
plt.xticks(rotation=45)  # Rotating x-axis labels for better readability

plt.tight_layout()  # Ensures no content is cut off in the figure
plt.show()  # Display the plot

### Step 4: Querying and Visualizing Location Data

Now, we will query the database to get the number of users per unique location visited and visualize it using a bar plot.

In [4]:
# %%
# Querying the database for the number of users per unique location visited
query_locations_per_user = "SELECT location_count, COUNT(user_id) as user_count FROM locations_per_user GROUP BY location_count ORDER BY location_count;"
df_locations_per_user = pd.read_sql_query(query_locations_per_user, engine)

# %%
# Visualizing the number of users by the unique number of locations they visited using a bar plot
plt.figure(figsize=(12, 6))
sns.barplot(data=df_locations_per_user, x='location_count', y='user_count', palette='viridis')

# Adding labels and title to the plot
plt.title("Number of Users by Unique Locations Visited")
plt.xlabel("Number of Unique Locations Visited")
plt.ylabel("Number of Users")
plt.xticks(rotation=0)  # Keeping x-axis labels horizontal

plt.tight_layout()  # Ensures the plot is displayed correctly
plt.show()

### Step 5: Handling Geospatial Data

Let's use `GeoPandas` to work with spatial data, including retrieving boundaries and merging them with the user data.

In [5]:
# %%
# Defining the target date for querying unique users by administrative unit
target_date = '2024-10-01'

# Query to retrieve the number of unique users per administrative unit (LAU) for the specified date
query_unique_users_per_admin_unit = f"""
SELECT lau_id, lau_name, unique_user_count
FROM unique_users_per_admin_unit
WHERE date = '{target_date}';
"""
# Running the query to retrieve the data into a DataFrame
df_population = pd.read_sql_query(query_unique_users_per_admin_unit, engine)

# Query to retrieve geographical boundaries of administrative units (LAUs)
query_boundaries = "SELECT lau_id, lau_name, geometry FROM local_admin_units;"
# Reading the spatial data (geometries) using GeoPandas
gdf_boundaries = gpd.read_postgis(query_boundaries, engine, geom_col='geometry')

# Merging the population data with the boundaries data using 'lau_id' as the key
gdf_boundaries = gdf_boundaries.merge(df_population, on="lau_id", how="left")

# Filling any missing unique user counts with 0 (if there are administrative units with no data)
gdf_boundaries['unique_user_count'] = gdf_boundaries['unique_user_count'].fillna(0)

### Step 6: Visualizing Geospatial Data

Now we will create a choropleth map to visualize unique user counts by administrative unit. The areas will be colored based on the number of unique users.

In [6]:
# %%
# Plotting a choropleth map to visualize unique user counts by administrative unit
fig, ax = plt.subplots(1, 1, figsize=(12, 10))
gdf_boundaries.plot(column='unique_user_count', cmap='viridis', linewidth=0.8, ax=ax, edgecolor='0.8', legend=True)

# Removing the axes for a cleaner map visualization
ax.set_axis_off()

# Displaying the choropleth map
plt.show()